In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-systems-triangular)

```{index} ! Julia; tril, ! Julia; triu
```

It's easy to get just the lower triangular part of any matrix using the `tril` function.
```{tip}
:class: dropdown
Use `tril` to return a matrix that zeros out everything above the main diagonal. The `triu` function zeros out below the diagonal.
```

In [2]:
A = rand(1.:9., 5, 5)
L = tril(A)

5×5 Matrix{Float64}:
 3.0  0.0  0.0  0.0  0.0
 9.0  8.0  0.0  0.0  0.0
 1.0  1.0  4.0  0.0  0.0
 4.0  5.0  8.0  7.0  0.0
 3.0  3.0  2.0  9.0  9.0

We'll set up and solve a linear system with this matrix.

In [3]:
b = ones(5)
x = FNC.forwardsub(L,b)

5-element Vector{Float64}:
  0.3333333333333333
 -0.25
  0.22916666666666669
 -0.13095238095238096
  0.16335978835978837

It's not clear how accurate this answer is. However, the residual should be zero or comparable to $\macheps$.

In [4]:
b - L * x

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

```{index} ! Julia; Pair, Julia; diagm
```

Next we'll engineer a problem to which we know the exact answer. Use `\alpha` <kbd>Tab</kbd> and `\beta` <kbd>Tab</kbd> to get the Greek letters.
```{tip}
:class: dropdown
The notation `0=>ones(5)` creates a `Pair`. In `diagm`, pairs indicate the position of a diagonal and the elements that are to be placed on it.
```

In [5]:
α = 0.3;
β = 2.2;
U = diagm( 0=>ones(5), 1=>[-1, -1, -1, -1] )
U[1, [4, 5]] = [ α - β, β ]
U

5×5 Matrix{Float64}:
 1.0  -1.0   0.0  -1.9   2.2
 0.0   1.0  -1.0   0.0   0.0
 0.0   0.0   1.0  -1.0   0.0
 0.0   0.0   0.0   1.0  -1.0
 0.0   0.0   0.0   0.0   1.0

In [6]:
x_exact = ones(5)
b = [α, 0, 0, 0, 1]

5-element Vector{Float64}:
 0.3
 0.0
 0.0
 0.0
 1.0

Now we use backward substitution to solve for $\mathbf{x}$, and compare to the exact solution we know already.

In [7]:
x = FNC.backsub(U,b)
err = x - x_exact

5-element Vector{Float64}:
 2.220446049250313e-16
 0.0
 0.0
 0.0
 0.0

Everything seems OK here. But another example, with a different value for $\beta$, is more troubling.

In [8]:
α = 0.3;
β = 1e12;
U = diagm( 0=>ones(5), 1=>[-1, -1, -1, -1] )
U[1, [4, 5]] = [ α - β, β ]
b = [α, 0, 0, 0, 1]

x = FNC.backsub(U,b)
err = x - x_exact

5-element Vector{Float64}:
 -4.882812499995559e-5
  0.0
  0.0
  0.0
  0.0

It's not so good to get 4 digits of accuracy after starting with 16! The source of the error is not hard to track down. Solving for $x_1$ performs $(\alpha-\beta)+\beta$ in the first row. Since $|\alpha|$ is so much smaller than $|\beta|$, this a recipe for losing digits to subtractive cancellation.